In [1]:
from typing import Hashable, List, NamedTuple

# Autómatas con pila

Un **autómata de pila** es una 6-ada $(Q, \Sigma, \Gamma, \delta, q_0, F)$
donde:
1. $Q$ es un conjunto finito de *estados*,
2. $\Sigma$ es el *alfabeto de entrada*,
3. $\Gamma$ es el *alfabeto de pila*,
4. $\delta: Q \times \Sigma_\epsilon \times \Gamma_\epsilon \to \mathcal{P}(Q 
   \times \Gamma_\epsilon)$ es la función de transición,
5. $q_0 \in Q$ es el *estado inicial*, y
6. $F \subseteq Q$ es el conjunto de *estados de aceptación*.

Equivalente, para nuestro curso, podemos expresar un autómata de pila
usando sólo
1. Un *programa* $P \subseteq Q \times \Sigma_\epsilon \times \Gamma_\epsilon 
   \times Q \times \Gamma_\epsilon$,
2. el *estado inicial* $q_0 \in Q$, y
3. el conjunto de *estados de aceptación* $F \subseteq Q$.

In [2]:
def A(m, n):
    assert m >= 0 and n >= 0
    if m == 0:
        return n + 1
    if n == 0:
        return A(m - 1, 1)
    return A(m - 1, A(m, n - 1))

In [3]:
A(4, 4)

RecursionError: maximum recursion depth exceeded in comparison

**Ejemplo**:
Un autómata de pila que reconoce el lenguaje $L = \left\{\texttt{0}^n
\texttt{1}^n\middle| n\in\mathbb{N}\right\}$

Cómo reconocer una palabra de L:
0(0(0(0(0(0(0(0()1)1)1)1)1)1)1)1

In [4]:
def reconocer_L(palabra):
    pila = ['$']
    bandera = False
    for simbolo in palabra:
        if not bandera and simbolo == '0':
            pila.append('0')
        elif not bandera and simbolo == '1' and pila[-1] == '0':
            bandera = True
            pila.pop()
        elif bandera and simbolo == '1' and pila[-1] == '0':
            pila.pop()
        else:
            return False
    return (pila[-1] == '$')

In [5]:
reconocer_L('0011')

True

In [6]:
Estado = Hashable
Simbolo = str

class Transicion(NamedTuple):
    estado: Estado
    entrada: Simbolo
    desapilar: Simbolo
    siguiente: Estado
    apilar: Simbolo

In [7]:
P = [
    Transicion(0, '',  '',  1, '$'),
    Transicion(1, '0', '',  1, '0'),
    Transicion(1, '1', '0', 2, ''),
    Transicion(2, '1', '0', 2, ''),
    Transicion(2, '',  '$', 3, ''),
]
q_0 = 0
F = [3]

**Ejercicio** Escribir un autómata de pila para reconocer el lenguaje de los
paréntesis bien balanceados.

In [8]:
P = [
    (0, '',  '',  1, '$'),
    (1, '(', '',  1, '('),
    (1, ')', '(', 1, '' ),
    (1, '',  '$', 2, '' )
]
q_0 = 0
F = [2]
M = (P, q_0, F)

## Simular autómatas de pila

In [9]:
import collections

In [10]:
def coincide_pila(S, retirar):
    return not retirar or (S and S[-1] == retirar)

def modificar_pila(S, retirar, apilar):
    T = S.copy()
    if retirar: T.pop()
    if apilar: T.append(apilar)
    return T

def ejecutar(M, w):
    (P, q_0, F) = M
    cola = collections.deque([(q_0, [], 0)])
    while cola:  # Mientras haya configuraciones por explorar...
        # Cada conf. tiene estado actual q, pila actual S, y la cantidad
        # i de símbolos consumidos de la entrada.
        q, S, i = cola.popleft()  # Siguiente configuración
        if q in F and w[i:] == '':
            return True  # ¿Configuración de aceptación?
        # Explorar posibles configuraciones siguientes:
        for estado, simbolo, retirar, siguiente, apilar in P:
            if q != estado or not coincide_pila(S, retirar): continue
            T = modificar_pila(S, retirar, apilar)
            if not simbolo:  # simbolo == epsilon
                cola.append((siguiente, T, i))  # Nueva configuración
            elif simbolo == w[i: i+1]:  # w[i] o epsilon si i >= len(w)
                cola.append((siguiente, T, i + 1))  # Nueva configuración
    return False

In [11]:
ejecutar(M, '(()(()))')

True

## Gramáticas libres de contexto

Una **gramática libre de contexto** es una 4-ada $(V, \Sigma, R, S)$ donde
1. $V$ es un conjunto finito de **variables**,
2. $\Sigma$ es un conjunto finito disjunto de $V$ de **terminales**,
3. $R$ es un conjunto finito de **reglas** tomadas de
   $V\times(V\cup\Sigma)^\star$,
4. $S\in V$ es la **variable inicial**.

**Notación**
- La regla $(V, \sigma_1\sigma_2\cdots\sigma_n)$ se denota por
  $V \to \sigma_1\sigma_2\cdots\sigma_n$.
- Si $(V, r_1), (V, r_2), \ldots, (V, r_m)$ son todas las reglas que inician
  con la misma variable $V$, entonces denotamos a este conjunto de reglas por
  $V \to r_1 | r_2 | \cdots | r_m$.
- Las reglas que inician con la variable inicial $S$ se escriben antes que las
  demás.

**Ejemplo**
Una gramática que produce cadenas $\mathtt{0}^n\mathtt{1}^n$:
$$\begin{align}
A &\to \texttt{0}A\texttt{1} | B \\
B &\to \texttt{#}
\end{align}$$

$A \Rightarrow \texttt{0}A\texttt{1} \Rightarrow \texttt{0}\texttt{0}A
\texttt{1}\texttt{1} \Rightarrow \texttt{0}\texttt{0}\texttt{0}A\texttt{1}
\texttt{1}\texttt{1} \Rightarrow \texttt{0}\texttt{0}\texttt{0}B\texttt{1}
\texttt{1}\texttt{1} \Rightarrow \texttt{0}\texttt{0}\texttt{0}\texttt{#}
\texttt{1}\texttt{1}\texttt{1}$

In [12]:
import random
def A():
    if random.getrandbits(1):
        return f'0{A()}1'
    return B()

def B():
    return '#'

In [21]:
A()

'0000#1111'

**Ejemplo** Una gramática para producir enunciados en español
$$\begin{align}
\textit{<enunciado>} &\to \textit{<sujeto>} \textit{<verbo>} \textit{<predicado>} \\
\textit{<sujeto>} &\to \textit{<sujeto_compuesto>} | \textit{<sujeto_compuesto>}\,\textit{<adjetivo>} \\
\textit{<sujeto_compuesto>} &\to \textit{<artículo>} \textit{<sustantivo>} \\
\textit{<artículo>} &\to \textbf{el} | \textbf{la} | \textbf{un} \\
\textit{<adjetivo>} &\to \textbf{dubitativo} | \textbf{rojo} | \textbf{chingón} | \textbf{grande} \\
\textit{<sustantivo>} &\to \textbf{caguama} | \textbf{borracho} | \textbf{programador} | \textbf{florero} \\
\textit{<verbo>} &\to \textbf{caguamea} | \textbf{banquetea} | \textbf{duerme} | \textbf{sueña} \\
\textit{<predicado>} &\to \textit{<preposición>}\textit{<sustantivo>}\\
\textit{<preposición>} &\to \textbf{con} | \textbf{sin} | \textbf{sobre} | \textbf{por}
\end{align}$$

$$\begin{align}
\mathit{<enunciado>} &\Rightarrow \textit{<sujeto>} \textit{<verbo>} \textit{<predicado>} \\
&\Rightarrow \textit{<sujeto_compuesto>}\textit{<adjetivo>} \textit{<verbo>} \textit{<predicado>} \\
&\Rightarrow \textit{<artículo>} \textit{<sustantivo>} \textit{<adjetivo>} \textit{<verbo>} \textit{<predicado>} \\
&\Rightarrow \textbf{la } \textit{<sustantivo>} \textit{<adjetivo>} \textit{<verbo>} \textit{<predicado>} \\
&\Rightarrow \textbf{la } \textbf{caguama } \textit{<adjetivo>} \textit{<verbo>} \textit{<predicado>} \\
&\Rightarrow \textbf{la } \textbf{caguama } \textbf{rojo } \textit{<verbo>} \textit{<predicado>} \\
&\Rightarrow \textbf{la } \textbf{caguama } \textbf{rojo } \textbf{caguamea } \textit{<predicado>} \\
&\Rightarrow \textbf{la } \textbf{caguama } \textbf{rojo } \textbf{caguamea } \textit{<preposición>}\textit{<sustantivo>}\\
&\Rightarrow \textbf{la } \textbf{caguama } \textbf{rojo } \textbf{caguamea } \textbf{sobre }\textit{<sustantivo>}\\
&\Rightarrow \textbf{la } \textbf{caguama } \textbf{rojo } \textbf{caguamea } \textbf{sobre }\textbf{banqueta }\\
\end{align}$$

**Ejemplo**
$$\begin{align}
E &\to E + T | T \\
T &\to T \times F | F \\
F &\to \texttt{(}E\texttt{)} | \texttt{a} \\
\end{align}$$

In [22]:
def E():
    if random.random() < 0.5:
        return f'{E()}+{T()}'
    return T()

def T():
    if random.random() < 0.3:
        return f'{T()}×{F()}'
    return F()

def F():
    if random.random() < 0.1:
        return f'({E()})'
    return 'a'

In [25]:
E()

'a×(a+a)+a×a'

## Equivalencia entre autómatas de pila y gramáticas libres de contexto

**Ejemplo** Convertir una gramática libre de contexto a un A.P.

In [16]:
P = [
    (0, '',  '',  1, '$'),
    # Inicio
    (1, '',  '',  2, 'E'),
    # E -> E + T
    (2, '',  'E', 3, 'T'),
    (3, '',  '',  4, '+'),
    (4, '',  '',  2, 'E'),
    # E -> T
    (2, '',  'E', 2, 'T'),
    # T -> T × F
    (2, '',  'T', 5, 'F'),
    (5, '',  '',  6, '×'),
    (6, '',  '',  2, 'T'),
    # T -> F
    (2, '',  'T', 2, 'F'),
    # F -> (E)
    (2, '',  'F', 7, ')'),
    (7, '',  '',  8, 'E'),
    (8, '',  '',  2, '('),
    # F -> a
    (2, '',  'F', 2, 'a'),
    # Terminales:
    (2, 'a', 'a', 2, ''),
    (2, ')', ')', 2, ''),
    (2, '(', '(', 2, ''),
    (2, '×', '×', 2, ''),
    (2, '+', '+', 2, ''),
    # Fin
    (2, '',  '$', 9, ''),
]
q_0 = 0
F = [9]
M = (P, q_0, F)

In [23]:
ejecutar(M, '(a+a)×a')

True

Para demostrar el regreso (que todo autómata de pila reconoce un lenguaje que
es producido por una gramática libre de contexto) tenemos que hacer unas
suposiciones:

1. Tiene un único estado de aceptación $q_{\mathit{accept}}$.
2. Vacía la pila antes de aceptar.
3. Cada trancisión o bien apila un símbolo o lo desapila, pero no puede
   apilar y desapilar al mismo tiempo.
   
*Idea de la demostración*
- $A_{p\,q}$ produce todas las cadenas que hacen que el autómata de pila
  pase del estado $p$ al estado $q$.

In [26]:
class Produccion(NamedTuple):
    variable: Simbolo
    substitucion: str
    
def gramatica_a_latex(producciones: List[Produccion]):
    gramatica = {}
    lineas = ['\\begin{align}']
    for variable, substitucion in producciones:
        gramatica.setdefault(variable, []).append(substitucion)
    
    for variable, substituciones in gramatica.items():
        derecha = '|'.join(substituciones)
        lineas.append(f'{variable} &\\rightarrow {derecha} \\\\')
    
    lineas.append('\\end{align}')
    return '\n'.join(lineas)

In [27]:
import itertools

In [28]:
def hacer_gramatica(M):
    def A(p, q):
        return f'A_{{{p}\\,{q}}}'
    P: List[Transicion]
    (P, q_0, F) = M
    
    Q = set(p.estado for p in P)
    Q.update(p.siguiente for p in P)
    Q.add(q_0)
    Q.update(F)
    Σ = set(p.entrada for p in P)
    Γ = set(p.desapilar for p in P) | set(p.apilar for p in P)
    
    G = []
    # CASO I
    for transicion_1, transicion_2 in itertools.product(P, repeat=2):
        if (x:=transicion_1.apilar) and transicion_2.desapilar == x:
            p, r = transicion_1.estado, transicion_1.siguiente
            s, q = transicion_2.estado, transicion_2.siguiente
            a, b = transicion_1.entrada, transicion_2.entrada
            G.append(Produccion(
                variable=A(p, q),
                substitucion=f'{a}{A(r, s)}{b}')
            )
    
    # CASO II
    for p, q, r in itertools.product(Q, repeat=3):
        G.append(Produccion(
            variable=A(p, q),
            substitucion=f'{A(p, r)}{A(r, q)}')
        )
    
    # Encargarse de los caminos que no consumen entrada:
    for p in Q:
        G.append(Produccion(variable=A(p, p), substitucion='\\epsilon'))
    
    return G

In [29]:
P = [
    Transicion(0, '',  '',  1, '$'),
    Transicion(1, '(', '',  1, '('),
    Transicion(1, ')', '(', 1, '' ),
    Transicion(1, '',  '$', 2, '' )
]
q_0 = 0
F = [2]
M = (P, q_0, F)

In [30]:
from IPython import display

In [31]:
display.display_latex(gramatica_a_latex(hacer_gramatica(M)), raw=True)

**Lema** Si $A_{p\,q}$ produce $x$, entonces $x$ hace que el A.P. $P$
pase del estado $p$ con pila vacía hacia el estado $q$ con pila vacía.

*Demostración* Por inducción sobre el número de pasos para derivar $x$
de $A_{p\,q}$.

**Caso base**: La derivación de $x$ requiere 1 paso.

Si la derivación tiene 1 paso entonces la *substitución* de la producción
no contiene variables. Entonces se trata de una producción $A_{p\,p}
\rightarrow \varepsilon$ y claramente se puede pasar del estado $p$ con
pila vacía al estado $p$ con pila vacía consumiendo $\varepsilon$ de la
entrada.

**Caso inductivo** Supongamos que esto es verdad para las derivaciones de
longitud a lo más $k \ge 1$ y demostramos que también es verdad para las
derivaciones de longitud $k + 1$.

Supongamos $A_{p\,q}\overset{*}{\Rightarrow} x$ tiene $k + 1$ pasos.
Entonces la primera derivación o bien es $A_{p\,q}\Rightarrow \texttt{a}
A_{r\,s}\texttt{b}$ o bien es $A_{p\,q}\Rightarrow A_{p\,r}A_{r\,q}$.

**CASO I** 
La primera derivación es $A_{p\,q}\Rightarrow \texttt{a} A_{r\,s} \texttt{b}$.

Entonces $x = \texttt{a} y \texttt{b}$ y además $A_{r\,s} \Rightarrow y$ en $k$
pasos.
Entonces por H.I. $A_{r\,s}$ hace que $P$ pase del estado $r$ con pila
vacía al estado $s$ con pila vacía.
Dado que existe la producción $A_{p\,q} \to \texttt{a} A_{r\,s}
\texttt{b}$ en la gramática, entonces existen dos transiciones
$t_1 = (p, \texttt{a}, \varepsilon, r, g)$ y $t_2 = (s, \texttt{b}, g, q,
\varepsilon)$ en el programa de $P$.

Si $P$ inicia con una pila vacía en el estado $p$ entonces leyendo $\texttt{a}$
podemos pasar al estado $r$ colocando $g$ en la pila.
Entonces leyendo $y$ pasamos de $r$ a $s$ y finalmente sacamos a $g$ de la pila
para pasar a $q$.
Por lo tanto $x$ hace que $P$ pase de $p$ con pila vacía a $q$ con pila vacía.

**CASO II**
La primera derivación es $A_{p\,q}\Rightarrow A_{p\,r}A_{r\,q}$.

Entonces $x=yz$ donde $A_{p\,r} \overset{*}{\Rightarrow} y$ y $A_{r\,q} 
\overset{*}{\Rightarrow} z$ en $k$ pasos o menos cada una.
Entonces por H.I. podemos pasar del estado $p$ con pila vacía al estado $r$
con pila vacía consumiendo $y$, y del estado $r$ al estado $q$ con pila vacía
consumiendo $z$.
Por lo tanto es posible pasar del estao $p$ con pila vacía al estado $q$ con
pila vacía consumiendo $yz = x$.

**Lema** Si $x$ hace que el A.P. $P$ pase del estado $p$ con pila vacía hacia
el estado $q$ con pila vacía entonces $A_{p\,q}$ produce $x$.

*Demostración* Por inducción sobre el número de pasos en el cómputo del A.P.

**Caso base**
El cómputo tiene 0 pasos. Entonces $p = q$ y $x = \varepsilon$ y la regla
$A_{p\,p} \to \varepsilon$ se encarga de llevar el autómata de $p$ a $p$ en
0 pasos.

**Caso inductivo**
Supongamos que $P$ pasa del estado $p$ a $q$ con pila vacía en $k + 1$ pasos,
entonces hay dos posibilidades: o bien la pila nunca se vacía durante las
trancisiones de $p$ a $q$ o bien se vacía al menos una vez.

**CASO I**
El símbolo que se apila en el primer paso debe ser el mismo que se desapila en
el último paso. Sea $t$ este símbolo, sean $a$ y $b$ los símbolos que se
consumen de la entrada en el primer y último paso, y sean $r$ y $s$ los estados
segundo y penúltimo. Entonces $(p, a, \varepsilon, r, t)$ y
$(s, b, t, q, \varepsilon)$ son transciciones de $P$ y entonces la regla
$A_{p\,q}\to \texttt{a} A_{r\,s}\texttt{b}$ pertenece a la gramática generada.

Sea $x = \texttt{a}y\texttt{b}$. Entonces $y$ puede llevar a $P$ del estado
$r$ al estado $s$ con pila vacía, es decir, sin tocar al símbolo $t$.
Entonces, quitando la primera y última transición tenemos un cómputo de $k - 1$
pasos, y por H.I. $A_{r\,s}\overset{*}{\Rightarrow} y$; por lo tanto,
$A_{p\,q}\overset{*}{\Rightarrow} x$.

**CASO II**
Sup. que la pila se vacía en algún punto del cómputo que nos hace pasar de $p$
a $q$. Entonces existe un estado intermedio $r$ donde la pila queda vacía.
Entonces el cómputo se puede dividir en la parte que va de $p$ a $r$ y de $r$
a $q$; ambas con menos de $k + 1$ pasos.
Entonces por H.I. $A_{p\,r}\overset{*}{\Rightarrow} y$, y
$A_{r\,q}\overset{*}{\Rightarrow} z$ de manera que $x=yz$, y dado que en la 
gramática tenemos $A_{p\,q}\Rightarrow A_{p\,r}A_{r\,q}$ entonces 
$A_{p\,q}\overset{*}{\Rightarrow} x$.